In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import import_ipynb
import DataLoader_P_speed
import utils
import time
import numpy as np

In [ ]:
class args():
    def __init__(self):
        self.base_out_features = 2048
        self.t_obs = 8
        self.batch_size = 20
        self.scene_shape = [3, 144, 256]
        self.base_shape = [2048, 9, 16]
        self.hidden_size = 128
        self.embedding_size = 64
        self.input_size = 2
        self.output_size = 2
        self.sample = True
        self.n_train_sequences = 12000
        self.n_val_sequences = 3000
        self.trainOrVal = 'train'
        args.citywalks = False
        
        #dataset
        self.jaad_dataset = '../../../../data/smailait-data/jaad/annotations'
        self.dtype = 'train'
        self.from_file = True
        self.file = '/data/smailait-data/train_speed.csv'
        self.val_file = '/data/smailait-data/val_speed.csv'
        self.seq_len = 4
        self.predicted_seq_len = 4
        self.crop = 0.3
        self.activity_h = 9
        self.activity_w = 16
        self.truncate = 0
        self.final_frame_offset = 0
        self.loader_workers = 20
        self.loader_shuffle = True
        self.pin_memory = False
        self.image_resize = [144, 256]
        self.image_size = [1080, 1920]
        self.device='cuda'
        
args = args()

In [ ]:
class P_LSTM(nn.Module):
    def __init__(self):
        super(P_LSTM, self).__init__()
        
        self.encoder = nn.LSTM(input_size=4, hidden_size=256)
        
        self.decoder = nn.LSTMCell(input_size=4, hidden_size=256)
        
        self.out = nn.Linear(in_features=256, out_features=4)
        
        self.activation = nn.Hardtanh(min_val=-100, max_val=100)
        
    def forward(self, input_):
        _, (h, c) = self.encoder(input_.permute(1,0,2))
        outputs = torch.tensor([], device='cuda')
        in_dec = input_[:,-1,:]
        h = h.squeeze(0)
        c = c.squeeze(0)
        for i in range(8):
            h, c = self.decoder(in_dec, (h, c))
            output = self.activation(self.out(h))
            outputs = torch.cat((outputs, output.unsqueeze(1)), dim = 1)
            in_dec = output.detach()
        return outputs

In [ ]:
net = P_LSTM().to('cuda')

In [ ]:
train, val, test = DataLoader_P_speed.data_loader(args)

In [ ]:
len(test)

In [ ]:
learning_rate = 0.0001
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, threshold = 1e-4, verbose=True)
criterion = nn.MSELoss()
train_scores = []
val_scores = []

In [ ]:
for epoch in range(100):
    avg_epoch_train_loss = 0
    avg_epoch_val_loss = 0
    ade = 0
    fde = 0
    aiou = 0
    fiou = 0
    counter = 0
    for idx, (obs, target, obs_p, target_p) in enumerate(train):
        counter += 1
        obs = obs.type(torch.float32).to(device='cuda')
        target = target.type(torch.float32).to(device='cuda')
        
        net.zero_grad()
        preds = net(obs)
        train_loss = criterion(preds, target)/10
        train_loss.backward()
        optimizer.step()
        
        avg_epoch_train_loss += float(train_loss)
        
    avg_epoch_train_loss/=counter
    train_scores.append(avg_epoch_train_loss)
    
    counter=0
    for idx, (obs, target, obs_p, target_p) in enumerate(val):
        counter+=1
        obs = obs.type(torch.float32).to(device='cuda')
        target = target.type(torch.float32).to(device='cuda')
        
        with torch.no_grad():
            preds = net(obs)
            val_loss = criterion(preds, target)/10
            preds_p = utils.speed2pos(preds, obs_p.to('cuda'), 20, 'cuda')
            ade += float(utils.ADE_c(preds_p, target_p[:,:-1,:].to('cuda')))
            fde += float(utils.FDE_c(preds_p, target_p[:,:-1,:].to('cuda')))
            aiou += float(utils.AIOU(preds_p, target_p[:,:-1,:].to('cuda')))
            fiou += float(utils.FIOU(preds_p, target_p[:,:-1,:].to('cuda')))
            
        avg_epoch_val_loss += float(val_loss)
        
    avg_epoch_val_loss/=counter
    val_scores.append(avg_epoch_val_loss)
    ade/=counter
    fde/=counter     
    aiou/=counter
    fiou/=counter
    
    scheduler.step(val_loss)
    
    print('e:', epoch, ' | train_loss: %.4f'% avg_epoch_train_loss, ' | val_loss: %.4f'% avg_epoch_val_loss, 
          '| ade: %.4f'% ade, '| fde: %.4f'% fde, '| aiou: %.4f'% aiou, '| fiou: %.4f'% fiou)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,20))
plt.plot(list(range(len(train_scores))), train_scores)
plt.plot(list(range(len(val_scores))), val_scores)
plt.show()

In [ ]:
avg_epoch_val_loss = 0
ade = 0
fde = 0
aiou = 0
fiou = 0
counter=0
for idx, (obs, target, obs_p, target_p) in enumerate(test):
    counter+=1
    obs = obs.type(torch.float32).to(device='cuda')
    target = target.type(torch.float32).to(device='cuda')

    with torch.no_grad():
        preds = net(obs)
        val_loss = criterion(preds, target)
        preds_p = utils.speed2pos(preds, obs_p.to('cuda'), 20, 'cuda')
        ade += float(utils.ADE_c(preds_p, target_p[:,:-1,:].to('cuda')))
        fde += float(utils.FDE_c(preds_p, target_p[:,:-1,:].to('cuda')))
        aiou += float(utils.AIOU(preds_p, target_p[:,:-1,:].to('cuda')))
        fiou += float(utils.FIOU(preds_p, target_p[:,:-1,:].to('cuda')))

    avg_epoch_val_loss += float(val_loss)

avg_epoch_val_loss/=counter
ade/=counter
fde/=counter     
aiou/=counter
fiou/=counter
print('val_loss: %.4f'% avg_epoch_val_loss, '| ade: %.4f'% ade, '| fde: %.4f'% fde, '| aiou: %.4f'% aiou, '| fiou: %.4f'% fiou)